In [2]:
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf

# Layer Functions

In [3]:
def fc_layer(input_tensor, output_neurons, name, doRelu=True):
    with tf.variable_scope(name):
        shape = input_tensor.get_shape().as_list()
        dim = 1
        for d in shape[1:]:
            dim *= d
        x = tf.reshape(input_tensor, [-1, dim])

        init = tf.random_normal_initializer(dtype=tf.float32, mean=0., stddev=0.1)

        weights = tf.get_variable(name="W", initializer=init, shape=[dim, output_neurons])
        biases = tf.get_variable(name="b", initializer=init, shape=[output_neurons])
        
        # Fully connected layer. Note that the '+' operation automatically
        # broadcasts the biases.
        fc = tf.nn.bias_add(tf.matmul(x, weights), biases)

        if doRelu:
            fc = tf.nn.relu(fc)

        return fc

# Construct Net

In [4]:
tf.reset_default_graph()

In [5]:
learning_rate = .0001
iters = 10000
batch_size = 1

In [6]:
input_nums = tf.placeholder(tf.float32, shape=[None, 2, 1], name='input_nums')
label = tf.placeholder(tf.float32, shape=[None, 1], name='label')

In [7]:
layer1 = fc_layer(input_nums, 32, 'layer1', doRelu=True)
layer2 = fc_layer(layer1, 16, 'layer2', doRelu=True)
output = fc_layer(layer2, 1, 'layer3', doRelu=False)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
loss = tf.losses.mean_squared_error(predictions=output,labels=label)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
opt = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# Train Net

In [11]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())

total_loss = 0.

for it in range(iters):

    # Input data
    train_data = np.random.uniform(low=-1., high=1., size=(batch_size,2,1))
    label_data = np.sum(train_data, axis=1)

    out_loss, _ = sess.run([loss, opt], feed_dict={input_nums: train_data, label: label_data})

    total_loss += out_loss

    if it % 1000 == 0 and it != 0:
        print (total_loss / it)

0.6108772372421266
0.5209443379059587
0.3880664544855127
0.29589996310700195
0.23816115466520774
0.1993006139127171
0.17151310751392074
0.1505558418205
0.13414554142983964


# Run Inference

In [12]:
test_nums = np.array([.3, -.7]).reshape(1,2,1)

In [13]:
test_out = sess.run(output, feed_dict={input_nums: test_nums})

In [14]:
test_out

array([[-0.4352274]], dtype=float32)

# Check Value of Weights

In [15]:
# Layer 1 weights - NOTE THAT YOU HAVE TO ADD ":0" FOR A TENSOR (not for an op)
weight_check = tf.get_default_graph().get_tensor_by_name("layer1/W:0")

In [17]:
print (sess.run(weight_check))

[[-0.05499791  0.34143427  0.04941327 -0.25879753  0.1387203  -0.01624228
   0.25707397 -0.12612686 -0.13855977  0.03373919 -0.32463023 -0.10652398
  -0.14723903  0.28885636  0.01198885  0.08861446 -0.24387743 -0.19036406
  -0.11006048 -0.2104132   0.01663758 -0.07826228  0.20885164 -0.33584917
   0.06990956  0.07641693 -0.19608274 -0.17648749  0.21840057 -0.03743926
   0.03898008 -0.16360632]
 [-0.00797489  0.11324963 -0.10633794  0.08113813  0.19876641  0.05272452
   0.28063893 -0.33201253  0.09563021  0.08126398 -0.16633065  0.18218221
  -0.09054985  0.11533346 -0.24464805  0.19892749 -0.25088573 -0.13433157
  -0.2891779  -0.22887714 -0.09445945 -0.21585678  0.27212805 -0.09458359
   0.00878388  0.26201278 -0.22439581 -0.06799366  0.05453108 -0.30448684
   0.3597803  -0.19976616]]


Good luck trying to figure that out

# Explore Graph

In [19]:
graph = tf.get_default_graph()
graph_def = graph.as_graph_def()

for node in graph_def.node:
    print (node.name)

input_nums
label
layer1/Reshape/shape
layer1/Reshape
layer1/W/Initializer/random_normal/shape
layer1/W/Initializer/random_normal/mean
layer1/W/Initializer/random_normal/stddev
layer1/W/Initializer/random_normal/RandomStandardNormal
layer1/W/Initializer/random_normal/mul
layer1/W/Initializer/random_normal
layer1/W
layer1/W/Assign
layer1/W/read
layer1/b/Initializer/random_normal/shape
layer1/b/Initializer/random_normal/mean
layer1/b/Initializer/random_normal/stddev
layer1/b/Initializer/random_normal/RandomStandardNormal
layer1/b/Initializer/random_normal/mul
layer1/b/Initializer/random_normal
layer1/b
layer1/b/Assign
layer1/b/read
layer1/MatMul
layer1/BiasAdd
layer1/Relu
layer2/Reshape/shape
layer2/Reshape
layer2/W/Initializer/random_normal/shape
layer2/W/Initializer/random_normal/mean
layer2/W/Initializer/random_normal/stddev
layer2/W/Initializer/random_normal/RandomStandardNormal
layer2/W/Initializer/random_normal/mul
layer2/W/Initializer/random_normal
layer2/W
layer2/W/Assign
layer2/W

In [21]:
for node in graph_def.node:
    if node.name == 'layer1/b':
        print (node)

name: "layer1/b"
op: "VariableV2"
attr {
  key: "_class"
  value {
    list {
      s: "loc:@layer1/b"
    }
  }
}
attr {
  key: "container"
  value {
    s: ""
  }
}
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: 32
      }
    }
  }
}
attr {
  key: "shared_name"
  value {
    s: ""
  }
}

